In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
sl_df = pd.read_stata('../Data/1967-2016_stateleg.dta')

In [ ]:
sl_df

In [ ]:
# df_18['OFFICENAME'].value_counts()

In [ ]:
# subset to only state leg results
mask = sl_df['state'].str.contains('Tennessee')
sleg = sl_df[mask]

In [ ]:
sleg.head()

In [ ]:
sleg['seatsup'].value_counts()

In [ ]:
sleg.loc[sleg['seatsup'] == 99]

# new column to group every row into either State House or Senate
for index, row in sleg.iterrows():
    if 'House' in row['OFFICENAME']:
        stateleg_18.loc[index, 'chamber'] = 'State House'
    elif 'Senate' in row['OFFICENAME']:
        stateleg_18.loc[index, 'chamber'] = 'State Senate'
    else:
        stateleg_18.loc[index, 'chamber'] = 'Other'

In [ ]:
for index, row in sleg.iterrows():    
    if row.sen >= 1:
        sleg.loc[index, 'office'] = 'State Senate'
    elif row.sen < 1:
        sleg.loc[index, 'office'] = 'State House'
    else:
        sleg.loc[index, 'office'] = 'Other'

In [ ]:
sleg.office.value_counts()

In [ ]:
sleg = sleg.loc[sleg['year'] >= 1980]

In [ ]:
sleg

In [ ]:
sleg.head(10)

In [ ]:
sleg = sleg[['year', 'sen', 'dvote', 'rvote', 'ovote', 'office']]

In [ ]:
sleg

In [ ]:
cols = sleg.columns.drop('office')

sleg[cols] = sleg[cols].astype('Int64')

In [ ]:
sleg['total_votes'] = sleg['dvote'] + sleg['rvote'] + sleg['ovote']

In [ ]:
sleg['Rep%'] = (sleg['rvote'] / sleg['total_votes'] * 100).round(1)
sleg['Dem%'] = (sleg['dvote'] / sleg['total_votes'] * 100).round(1)
sleg['Other%'] = (sleg['ovote'] / sleg['total_votes'] * 100).round(1)

In [ ]:
sleg = sleg.rename(columns={'dvote': 'Dem', 'rvote': 'Rep', 'ovote': 'Other'})

In [ ]:
sleg = sleg[['year', 'Dem', 'Rep', 'Other', 'office', 'total_votes']]

In [ ]:
sleg_year = sleg.groupby(['year', 'office']).agg({'Dem': 'sum', 'Rep': 'sum', 'Other': 'sum', 'total_votes': 'sum'}).reset_index()

In [ ]:
sleg_year.head()

In [ ]:
sleg_year['Rep%'] = (sleg_year['Rep'] / sleg_year['total_votes'] * 100).round(1)
sleg_year['Dem%'] = (sleg_year['Dem'] / sleg_year['total_votes'] * 100).round(1)
sleg_year['Other%'] = (sleg_year['Other'] / sleg_year['total_votes'] * 100).round(1)

In [ ]:
sleg_year = sleg_year[['year', 'office', 'total_votes', 'Rep', 'Dem', 'Other', 'Rep%', 'Dem%', 'Other%']]

In [ ]:
sleg_year

In [ ]:
stateleg_18 = pd.read_csv('../Data/clean_stateleg_18.csv')
stateleg_20 = pd.read_csv('../Data/clean_stateleg_20.csv')

In [ ]:
stateleg_18.head(3)

In [ ]:
stateleg_20.head(3)

In [ ]:
stateleg_master = pd.concat([sleg_year, stateleg_18, stateleg_20])
stateleg_master

In [ ]:
# stateleg_master.to_csv("clean_stateleg_statewide.csv", index=False)

In [ ]:
# create 3 new columns for votes by party (Rep, Dem, Other)
for index, row in stateleg_18.iterrows():
    if row.PARTY1 == "Republican":
        stateleg_18.loc[index, 'Rep'] = stateleg_18.loc[index, 'PVTALLY1']
    elif row.PARTY1 == "Democratic":
        stateleg_18.loc[index, 'Dem'] = stateleg_18.loc[index, 'PVTALLY1']
    else:
        stateleg_18.loc[index, 'Other'] = stateleg_18.loc[index, 'PVTALLY1']

In [ ]:
for index, row in stateleg_18.iterrows():
    if row.PARTY2 == "Republican":
        stateleg_18.loc[index, 'Rep'] = stateleg_18.loc[index, 'PVTALLY2']
    elif row.PARTY2 == "Democratic":
        stateleg_18.loc[index, 'Dem'] = stateleg_18.loc[index, 'PVTALLY2']
    else:
        stateleg_18.loc[index, 'Other'] = stateleg_18.loc[index, 'PVTALLY2']

In [ ]:
for index, row in stateleg_18.iterrows():
    if row.PARTY3 == "Democratic":
        stateleg_18.loc[index, 'Dem'] = stateleg_18.loc[index, 'PVTALLY3']
    else:
        stateleg_18.loc[index, 'Other'] = stateleg_18.loc[index, 'PVTALLY3']

In [ ]:
# eliminate unnecessary columns
stateleg_18 = stateleg_18[['Rep', 'Dem', 'Other', 'chamber']]

In [ ]:
stateleg_18.head()

In [ ]:
# change any floats to int
cols = stateleg_18.columns.drop('chamber')

stateleg_18[cols] = stateleg_18[cols].astype('Int64')

In [ ]:
# create total votes column
stateleg_18['total_votes'] = stateleg_18['Dem'].fillna(0) + stateleg_18['Rep'].fillna(0) + stateleg_18['Other'].fillna(0)

In [ ]:
stateleg_18.info()

In [ ]:
stateleg_18.head()

In [ ]:
# group by chamber, agg vote sums for Rep, Dem, Other
sl_18_clean = stateleg_18.groupby('chamber').agg({'Rep': 'sum', 'Dem': 'sum', 'Other': 'sum', 'total_votes': 'sum'}).reset_index()

In [ ]:
sl_18_clean

In [ ]:
# create new columns for vote % (Rep, Dem, Other)
sl_18_clean['Rep%'] = (sl_18_clean['Rep'] / sl_18_clean['total_votes'] * 100).round(1)
sl_18_clean['Dem%'] = (sl_18_clean['Dem'] / sl_18_clean['total_votes'] * 100).round(1)
sl_18_clean['Other%'] = (sl_18_clean['Other'] / sl_18_clean['total_votes'] * 100).round(1)

In [ ]:
# create year column
sl_18_clean['year'] = '2018'

In [ ]:
sl_18_clean = sl_18_clean.rename(columns={'chamber': 'office'})

In [ ]:
sl_18_clean

In [ ]:
sl_18_clean = sl_18_clean[['year', 'office', 'total_votes', 'Rep', 'Dem', 'Other', 'Rep%', 'Dem%', 'Other%']]

In [ ]:
sl_18_clean

In [ ]:
# sl_18_clean.to_csv("clean_stateleg_18.csv", index=False)